In [3]:
# Read the bronze table
df = spark.read.table("trips_raw")
print(f"Raw rows loaded: {df.count():,}")

StatementMeta(, 20696812-f948-4ec7-8006-712fa740f234, 5, Finished, Available, Finished)

Raw rows loaded: 68,211


In [2]:
from pyspark.sql import functions as F

df_silver = (df
    .filter((F.col("total_amount") > 0) & (F.col("trip_distance") > 0))
    .select(
        F.col("VendorID").alias("vendor_id"),
        F.col("tpep_pickup_datetime").alias("pickup_datetime"),
        F.col("tpep_dropoff_datetime").alias("dropoff_datetime"),
        "passenger_count",
        "trip_distance",
        F.col("PULocationID").alias("pickup_location_id"),
        F.col("DOLocationID").alias("dropoff_location_id"),
        "fare_amount",
        "total_amount"
    )
)
print("Transformations applied")

StatementMeta(, 20696812-f948-4ec7-8006-712fa740f234, 4, Finished, Available, Finished)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `tpep_pickup_datetime` cannot be resolved. Did you mean one of the following? [`lpep_pickup_datetime`, `lpep_dropoff_datetime`, `ehail_fee`, `tip_amount`, `trip_distance`].;
'Project [VendorID#756L AS vendor_id#1140L, 'tpep_pickup_datetime AS pickup_datetime#1141, 'tpep_dropoff_datetime AS dropoff_datetime#1142, passenger_count#763, trip_distance#764, PULocationID#761L AS pickup_location_id#1143L, DOLocationID#762L AS dropoff_location_id#1144L, fare_amount#765, total_amount#772]
+- Filter ((total_amount#772 > cast(0 as double)) AND (trip_distance#764 > cast(0 as double)))
   +- SubqueryAlias spark_catalog.lh_taxi_demo.trips_raw
      +- Relation spark_catalog.lh_taxi_demo.trips_raw[VendorID#756L,lpep_pickup_datetime#757,lpep_dropoff_datetime#758,store_and_fwd_flag#759,RatecodeID#760,PULocationID#761L,DOLocationID#762L,passenger_count#763,trip_distance#764,fare_amount#765,extra#766,mta_tax#76

In [ ]:
# Write as Delta table in silver layer
df_silver.write.format("delta").mode("overwrite").saveAsTable("silver.fact_trips")
print("Silver table 'silver.fact_trips' created successfully!")

StatementMeta(, 20696812-f948-4ec7-8006-712fa740f234, -1, Cancelled, , Cancelled)

In [ ]:
row_count = spark.table("silver.fact_trips").count()
assert row_count > 2_800_000, f"Only {row_count} rows — something wrong!"
print(f"TEST PASSED: {row_count:,} rows in silver.fact_trips")

StatementMeta(, 20696812-f948-4ec7-8006-712fa740f234, -1, Cancelled, , Cancelled)